In [3]:
# xóa VPNT-AI-Hackathon
!rm -rf VPNT-AI-Hackathon
# xóa kb_bge_m3_index.pkl
!rm -rf kb_bge_m3_index.pkl

In [4]:
ls

In [5]:
!git clone https://github.com/VuThanhLam124/VPNT-AI-Hackathon.git

Cloning into 'VPNT-AI-Hackathon'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 1), reused 25 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 16.14 MiB | 39.82 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
ls

VPNT-AI-Hackathon/


In [7]:
cd VPNT-AI-Hackathon

/kaggle/working/VPNT-AI-Hackathon


In [8]:
ls

AInicorns_TheBuilder_public_v1.1/  kb_vnpt_embedding_index.pkl
api_client.py                      LICENSE
build_index.py                     predict.py
data_utils.py                      README.md
Dockerfile                         requirements.txt
generate_simple.py                 rule.txt
Huong_dan_nop_bai_Track2.txt       Tai_lieu_API_LLM_Embedding.txt
inference.sh*                      test.ipynb


In [9]:
# check GPU status
!nvidia-smi

Mon Dec 15 10:27:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Kaggle: embed JSONL bằng BAAI/bge-m3 và (tuỳ chọn) load Qwen3-8B để inference local.
# Lưu ý: cần bật Internet trong Kaggle để download model từ Hugging Face (hoặc upload weights như Kaggle Dataset).

!pip -q install -U sentence-transformers transformers accelerate bitsandbytes

import json
import os
import pickle
from pathlib import Path
from typing import Dict, Iterator, List

import numpy as np
from tqdm.auto import tqdm

JSONL_ROOT = Path("/kaggle/input")
BGE_MODEL_ID = "BAAI/bge-m3"
MAX_CHUNK_CHARS = 800
MIN_TEXT_CHARS = 20
LIMIT_DOCS = 0  # 0 = không giới hạn
BGE_BATCH_SIZE = int(os.getenv("BGE_BATCH_SIZE", "32"))
OUT_PATH = Path("/kaggle/working/kb_bge_m3_index.pkl")

jsonl_paths = sorted(JSONL_ROOT.rglob("*.jsonl"))
if not jsonl_paths:
    raise FileNotFoundError(f"Không tìm thấy *.jsonl trong {JSONL_ROOT}")

print("Found JSONL files:")
for p in jsonl_paths[:50]:
    print("-", p)
if len(jsonl_paths) > 50:
    print(f"... (+{len(jsonl_paths) - 50} files)")


def iter_json_objects_from_mixed_jsonl(path: Path) -> Iterator[Dict]:
    raw = path.read_text(encoding="utf-8-sig")

    # 1) Try full JSON array/object
    try:
        s = raw.strip()
        if s.startswith("[") or s.startswith("{"):
            obj = json.loads(s)
            if isinstance(obj, list):
                for it in obj:
                    if isinstance(it, dict):
                        yield it
                return
            if isinstance(obj, dict):
                yield obj
                return
    except Exception:
        pass

    # 2) Try JSONL line-by-line (allow trailing comma)
    yielded_any = False
    for line in raw.splitlines():
        s = line.strip()
        if not s:
            continue
        s = s.rstrip("\\").rstrip()
        if s.endswith(","):
            s = s[:-1].rstrip()
        try:
            obj = json.loads(s)
        except Exception:
            continue
        if isinstance(obj, dict):
            yielded_any = True
            yield obj
    if yielded_any:
        return

    # 3) Fallback: sequential JSON decoder
    decoder = json.JSONDecoder()
    i = 0
    n = len(raw)
    while i < n:
        while i < n and raw[i] in " \t\r\n,\\":
            i += 1
        if i >= n:
            break
        if raw[i] not in "[{":
            i += 1
            continue
        try:
            obj, end = decoder.raw_decode(raw, i)
        except Exception:
            i += 1
            continue
        i = end
        if isinstance(obj, dict):
            yield obj
        elif isinstance(obj, list):
            for it in obj:
                if isinstance(it, dict):
                    yield it


def to_text(obj: Dict) -> str:
    if not isinstance(obj, dict):
        return ""
    if isinstance(obj.get("text"), str) and obj.get("text").strip():
        return obj["text"].strip()
    if "question" in obj and "answer" in obj:
        q = str(obj.get("question") or "").strip()
        a = str(obj.get("answer") or "").strip()
        if q:
            return f"Hỏi: {q}\nĐáp: {a}".strip()
    # fallback: key-value summary
    parts = []
    for k, v in obj.items():
        if v is None:
            continue
        vs = str(v).strip()
        if not vs:
            continue
        parts.append(f"{k}: {vs}")
    return ". ".join(parts).strip()


docs: List[Dict] = []
texts: List[str] = []

for path in tqdm(jsonl_paths, desc="Loading JSONL"):
    src = str(path)
    for obj in iter_json_objects_from_mixed_jsonl(path):
        t = to_text(obj)
        if not t or len(t) < MIN_TEXT_CHARS:
            continue
        if MAX_CHUNK_CHARS and len(t) > MAX_CHUNK_CHARS:
            t = t[:MAX_CHUNK_CHARS]
        docs.append({"text": t, "source": src})
        texts.append(t)
        if LIMIT_DOCS and len(texts) >= LIMIT_DOCS:
            break
    if LIMIT_DOCS and len(texts) >= LIMIT_DOCS:
        break

print("Docs:", len(docs))

from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("BGE device:", device)
embedder = SentenceTransformer(BGE_MODEL_ID, device=device)
try:
    embedder.max_seq_length = 512
except Exception:
    pass

emb = embedder.encode(
    texts,
    batch_size=BGE_BATCH_SIZE,
    show_progress_bar=True,
    normalize_embeddings=True,
)
emb = np.asarray(emb, dtype=np.float32)
print("Embeddings:", emb.shape, emb.dtype)

with open(OUT_PATH, "wb") as f:
    pickle.dump(
        {
            "backend": "bge",
            "model_name": BGE_MODEL_ID,
            "docs": docs,
            "embeddings": emb,
        },
        f,
    )
print("Saved:", OUT_PATH)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 128.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.

Loading JSONL:   0%|          | 0/14 [00:00<?, ?it/s]

Docs: 4635


2025-12-15 09:45:22.498851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765791922.663636     115 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765791922.712431     115 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/145 [00:00<?, ?it/s]

Embeddings: (4635, 1024) float32
Saved: /kaggle/working/kb_bge_m3_index.pkl
Load/run Qwen3-8B failed: OSError Qwen/Qwen3-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission 


In [20]:
USE_QWEN = True
QWEN_MODEL_ID = "Qwen/Qwen3-8B-Base"

if USE_QWEN:
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

    if not torch.cuda.is_available():
        print("CUDA chưa sẵn sàng -> bỏ qua load Qwen3-8B")
    else:
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        try:
            tok = AutoTokenizer.from_pretrained(QWEN_MODEL_ID, trust_remote_code=True)
            llm = AutoModelForCausalLM.from_pretrained(
                QWEN_MODEL_ID,
                device_map="auto",
                quantization_config=bnb_cfg,
                trust_remote_code=True,
            )
            llm.eval()
            print("Loaded:", QWEN_MODEL_ID)

            # demo: hỏi 1 câu mẫu
            sample_q = "Khỉ nào được dùng phổ biến nhất để sản xuất vắc-xin bại liệt ở Việt Nam?"
            qv = embedder.encode([sample_q], batch_size=1, show_progress_bar=False, normalize_embeddings=True)
            qv = np.asarray(qv, dtype=np.float32)[0]
            sims = emb @ qv
            top_idx = np.argsort(-sims)[:4]
            ctx = "\n\n---\n\n".join(docs[int(i)]["text"] for i in top_idx)

            prompt = (
                "Bạn là hệ thống trả lời trắc nghiệm. Chỉ trả về đáp án ngắn gọn.\n"
                "Dựa trên context, trả lời câu hỏi.\n\n"
                f"<context>\n{ctx}\n</context>\n\n"
                f"<question>\n{sample_q}\n</question>\n\n"
                "Trả lời:"
            )
            inputs = tok(prompt, return_tensors="pt")
            inputs = {k: v.to(llm.device) for k, v in inputs.items()}
            out = llm.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False,
                temperature=0.0,
            )
            ans = tok.decode(out[0], skip_special_tokens=True)
            print(ans[len(prompt):].strip())
        except Exception as e:
            print("Load/run Qwen3-8B failed:", type(e).__name__, str(e)[:200])


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Loaded: Qwen/Qwen3-8B-Base
Khỉ Rhesus


In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B-Base"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switch between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)
